# Game Simulation Pipeline

This notebook walks through the process of building a basketball game simulator with the objective of  The code here is optimized for a Monte Carlo simulation with modularized functions for quick replacement of deterministic methods with stochastic methods. 

__________

In [14]:
from dataclasses import dataclass, field
from typing import List, Dict
import random

#### Object Oriented Approach

Optimizing data organization is critical for efficient handling of Monte Carlo iterations and stochastic modeling. We want to balance structure with memory and computational efficiency. I have decided to take an object oriented approach, breaking down the simulation task into the following classes:

- **Simulation**: Handles the iterations of the Monte Carlo simulation along with aggregating the results from each game simulation.
- **Game**: Handles the game context for a single game simulation - clock, posession, team fouls, timeouts, etc.
- **PlayerGameState**: Handles the *mutable* in-game stats for a player - points, assists, rebounds, personal fouls, etc.
- **PlayerProfile**: Handles the *immutable* historical data retrieved from a database at the start of the simulation.

The separation of the mutable data from the immutable data is crutial for minimizing the memory required to run thousands of simulations of the same game. Here, a player profile will be generated once at the start of the simulation. Empty PlayerGameState objects will be initilized for tracking in-game stats and reference the same player profile objects afor any read-only data required from the player. This approach allows the Player profiles to be established only once, instead of requiring deep copies or clones to create new player profiles for each simulation.

### Simulating a Possession

A baseketball posession can be broken into a series of decisions that make up a decision tree. Some of these decisions are offensively oriented (shot taken), defensive oriented (foul before shot), or require interactions between both (rebounds).

We start each possession under the assumption that two things can happen: Offense attempts to run a play or Defense commits a foul to prevent that play from executing. We categorize this differently from a foul that occurs during the play execution (shooting foul).

In [ ]:
@dataclass
class LivePlayerState:
    points: int = 0
    assists: int = 0
    rebounds: int = 0 
    turnovers: int = 0
    minutes: float = 0.0
    shots: int = 0
    FT_taken: int = 0
    FT_made: int = 0
    two_pt_shots_taken: int = 0 # Need shots taken per minute? Standardized 
    two_pt_shots_made: int = 0
    three_pt_shots_taken: int = 0
    three_pt_shots_made: int = 0
    fg_percent: float = 0.0
    fouls: int = 0

class PlayerProfile:
    def __init__(self, params):
        self.name: str = params['name']
        self.number: int = params['j_number']
        self.team: str = params['team']
        self.fg_pct: float = params['fg_pct'] # Overall Shooting Percentage
        self.two_pt_pct: float = params['two_pt_pct'] # Two Point Shooting Percetage
        self.three_pt_pct: float = params['three_pt_pct'] # Three Point Shooting Percentage
        self.FT_pct: float = params['FT_pct'] # Free Throw Shooting Percentage
        self.FT_pg: float = params['FT_pg'] # Free Throws per Game
        self.pts_pg: float = params['pts_pg'] # Points per Game 
        self.ast_pg: float = params['ast_pg'] # Assists per Game
        self.off_reb_pg: float = params['off_reb_pg'] # Offensive Rebounds per Game **** This should really be standardized to rebounds per minute or possession
        self.def_reb_pg: float = params['def_reb_pg'] # Defensive Rebounds per Game 
        self.min_pg: float = params['min_pg'] # Minutes per Game
        self.shot_pg: float = params['shot_pg'] # Total Shots per Game
        self.to_pg: float = params['to_pg'] # Turnovers per Game
        self.to_prob: float = params['to_prob'] # Individual probability of a **** turnover per possession ****
        self.foul_prob: float = params['foul_prob'] # This has to be probability of ***** foul per possession *****
        self.two_pt_permin: float = params['two_pt_permin'] # Two point shots taken per minute
        self.three_pt_permin: float = params['three_pt_permin'] # Three point shots taken per minute

    def __str__(self):
        print(f"Player Profile Object: {self.name}")

class PlayerGameState:
    def __init__(self, profile):
        self.name: str = ""
        self.team: str = ""
        self.historical_stats: PlayerProfile = profile
        self.live_stats: LivePlayerState = LivePlayerState()

    def shot_taken(self, shot_type):
        valid_shots = ('1','2','3')
        if shot_type not in valid_shots:
            raise ValueError(f"Invalid shot type: '{shot_type}'. Expected '1','2', or '3'.")

        if shot_type == '2':
            return not bool(random.binomial(1, self.historical_stats.two_pt_pct)) # predicting below thresh returns True
        elif shot_type == '3':
            return not bool(random.binomial(1, self.historical_stats.three_pt_pct))
        else: # shot_type == 1 Free Throw
            return not bool(random.binomial(1, self.historical_stats.FT_pct))


In [ ]:
@dataclass
class LiveTeamState:
    name: str = ""
    players: List[PlayerGameState] = field(default_factory=list)
    start_lineup: List[PlayerGameState] = field(default_factory=list)
    live_lineup: List[PlayerGameState] = field(default_factory=list)
    points: int = 0
    team_fouls: int = 0
    timeouts: int = 5
    agg_foul_prob: float = 0.0
    agg_turnover_prob: float = 0.0
    agg_FT_pg: float = 0.0
    norm_foul_dist: List[float] = field(default_factory=list)
    norm_turnover_dist: List[float] = field(default_factory=list)
    norm_freethrows_dist: List[float] = field(default_factory=list)
    agg_2ptshots_per_minute: float = 0.0
    norm_2ptshooting_dist: List[float] = field(default_factory=list)
    agg_3ptshots_per_minute: float = 0.0
    norm_3ptshooting_dist: List[float] = field(default_factory=list)
    shot_selection_2pt: float = 0.0

class Game:
    def __init__(self, HomeTeam:str, AwayTeam:str):
        self.clock = 12*60
        self.quarter = 1
        self.HT = LiveTeamState(name = HomeTeam)
        self.AT = LiveTeamState(name = AwayTeam)
        self.OFF = self.HT #default 
        self.DEF = self.AT #default
        self.possession_arrow = None
        self.possession_data = { # This is an option for bulk game updates, instead of handling updates dynamically as events occur during posssession.
            'foul_commited': {},
            'points': {},
            'assists': {},
            'rebounds': {},
            'turnover': {},
            'shot_clock': 0,
        }

    def __str__(self):
        print(f"{self.AT} vs. {self.HT}")

    def simulate(self):
        self.start_game()
        while (self.clock > 0) & (self.quarter <= 4):
            if self.timeout_called():
                pass # Determine whether a timeout was called before the posession even started. Same with Lineup changes
            posession_data = self.simulate_posession()
            self.update_game(posession_data)
    
    def start_game(self):
        '''
        This function performs the following tasks:
        1. Determines first possession (tipoff) and posession arrow
        2. Establishes starting lineup
        3. Establishes the player stat distribution that the Monte Carlo simulation will sample from
        '''
        self.rotate_lineup()
        if self.tipoff():
            self.OFF = self.HT
            self.DEF, self.possession_arrow = self.AT
        else:
            self.OFF = self.AT
            self.DEF, self.possession_arrow = self.HT
        
        pass
    
    def simulate_possession(self):
        '''The return from this function should have all the relevant data for the possession in a standardized dictionary format'''

        foul_committed, free_throws_shot, rebound_opp = self.non_shooting_foul()
        if foul_committed:
            if free_throws_shot:
                if rebound_opp: 
                    if self.rebound(): #True = possession change
                        return # Foul committed, free throws attempted, final free throw missed, defensive rebound, possession change
                    return # Foul committed, free throws attempted, final free throw missed, offensive rebound, possession stays
                return # Foul committed, free throws attempted, final free throw made, possession change
            return # Foul committed, no free throws attempted, possession stays

        if foul_committed: # Don't look for what would keep the function going, look for what would break it as early as possible
            if not free_throws_shot: return # Foul committed, no free throws attempted, possession stays
            if not rebound_opp: return # Foul committed, free throws attempted, final free throw made, possession change
            if not self.rebound(): return # Foul committed, free throws attempted, final free throw missed, offensive rebound, possession stays
            return # Foul committed, free throws attempted, final free throw missed, defensive rebound, possession change
        

        if self.turnover():
            return # Turnover committed, possession change
        
        rebound_opp = self.offensive_play() # offensive_play() handles the shot selection and free throws if necessary. Returns rebound opportunity
        if not rebound_opp: return # Shot made (or free throw made), possession change
        if not self.rebound(): return # Shot missed, offensive rebound, possession stays
        return # Shot missed, defensive rebound, possession change
        
                
    def turnover(self):
        '''This function evaluates whether a turnover was committed during the possession.'''
        if bool(random.binomial(1, self.OFF.agg_turnover_prob)):
            player_TO_committed = random.choice(self.OFF.live_lineup, self.OFF.norm_turnover_dist)
            player_TO_committed.live_stats.turnovers += 1
            return True
        return False
        
    def non_shooting_foul(self):
        ''' 
        This function will most likely use an XGBoost model or a probability distribution to determine 
        whether a non-shooting foul interrupted the possession before a shot was taken or a turnover committed.
        For now it is represented as a static probability.'''

        if bool(random.binomial(1, self.DEF.agg_foul_prob)):
            foul_committed = True
            player_foulcommitted = random.choice(self.DEF.live_lineup, 1, p=self.DEF.norm_foul_dist)
            player_foulcommitted.live_stats.fouls += 1
            player_foulearned = random.choice(self.OFF.live_lineup, 1, p=self.OFF.norm_freethrows_dist)
            
            if self.DEF.team_fouls < 6:
                free_throws_shot, rebound_opp = False
                return foul_committed, free_throws_shot, rebound_opp
            elif (self.DEF.team_fouls >=6) & (self.DEF.team_fouls <=9):
                FT_type = 'one-one'
            else: # More than 9 team fouls
                FT_type = '2'

            rebound_opp = self.shoot_free_throws(player=player_foulearned, type = FT_type)
            free_throws_shot = True
            return foul_committed, free_throws_shot, rebound_opp
        
        foul_committed = False
        return foul_committed, None, None 
    
    def shoot_free_throws(self, player, type):
        '''This function will simulate as many free throws as needed, either, 1, 2, or 3 shots'''
        FT_type_map = {'1':1, '2':2, '3':3}
        if type == 'one-one':
            player.live_stats.FT_taken += 1

            if not player.shot_taken('1'): 
                return True
            
            player.live_stats.FT_taken += 1
            player.live_stats.points += 1
            if not player.shot_taken('1'): 
                return True
            
            player.live_stats.points += 1
            player.live_stats.FT_made += 1
            return False # Returning the "Rebounding Opportunity" which is false when the last shot is made

        
        if type not in FT_type_map.keys():
            raise ValueError(f"Unknown free throw type passed to shoot_free_throws function: '{type}'.")
        
        rebound_opp = None
        for _ in range(FT_type_map['type']):
            player.live_stats.FT_taken += 1
            if player.shot_taken('1'):
                player.live_stats.points += 1
                player.live_stats.FT_made += 1
                rebound_opp = False # No rebounding oppoortunity (False) because shot was made
            rebound_opp = True

        return rebound_opp

        
    def offensive_play(self):
        '''
        This function executes the decision tree if an offensive play is ran (shot is taken)
        '''
        shot_result = None
        if bool(random.binomial(1, self.OFF.shot_selection_2pt)): # True (> thresh) = 3pt shot, False (< thresh) = 2pt shot
            shot_selection = '3'
            shooter = random.choice(self.OFF.live_lineup, self.OFF.norm_3ptshooting_dist)
            shooter.live_stats.three_pt_shots_taken += 1
            if shooter.shot_taken('3'):
                shooter.live_stats.three_pt_shots_made += 1
                shooter.live_stats.points += 3 
                shot_result = True
            else: shot_result = False
        else: 
            shot_selection = '2'
            shooter = random.choice(self.OFF.live_lineup, self.OFF.norm_2ptshooting_dist)
            shooter.live_stats.two_pt_shots_taken += 1
            if shooter.shot_taken('2'):
                shooter.live_stats.two_pt_shots_made += 1
                shooter.live_stats.points += 2
                shot_result = True
            else: shot_result = False
        
        free_throws, rebound_opp = self.shooting_foul(shot_selection, shot_result, player=shooter) # Return the rebounding opportunity
        if free_throws:
            return rebound_opp
        else:
            return not shot_result
                
        
    
    def shooting_foul(self, shot_selection, shot_result, player):
        if bool(random.binomial(1, self.DEF.agg_foul_prob)):
            free_throws = True
            if shot_result:
                rebound_opp = self.shoot_free_throws('1')
            else:
                rebound_opp = self.shoot_free_throws(player=player, type=shot_selection)
        else: 
            free_throws = False
            rebound_opp = False
        
        return free_throws, rebound_opp
            

    
    def rebound(self):
        '''This function determines who would get a rebound based on the offensive and defensive players on the court'''
        
        active_players  = self.DEF.live_lineup + self.OFF.live_lineup
        def_rebounds = [player['def_reb_pg'] for player in self.DEF.live_lineup]
        all_ind_rebounds = def_rebounds + [player['off_reb_pg'] for player in self.OFF.live_lineup]
        total_rebounds = sum(all_ind_rebounds)
        all_ind_prob = [x / total_rebounds for x in all_ind_rebounds]
        rebounder = random.choice(active_players, 1, p = all_ind_prob)
        rebounder.live_stats.rebounds += 1
        if rebounder in self.DEF.live_lineup:
            possession_change = True
        else: possession_change = False
        return possession_change
    

    def rotate_lineup(self, team):
        '''
        This function will require recalculating probabilities depending on which lineup is on the floor.
        For example, when players are subbed into the game, there is a new likelihood of a foul and turnover being committed based on the lineup.
        These new probabilities will be updated as game state parameters'''

        valid_teams = ('HT', 'AT')
        if team not in valid_teams:
            raise ValueError(f"Invalid team passed to the lineup rotation: {team}. Expected 'HT' or 'AT'.")

        if team == 'HT':
            team = self.HT
        else:
            team == self.AT

        # Update the live_lineup parameter
        # ???? How to determine when a rotation is needed / who gets subbed in and out

        # Probabilities of each active player of committing a foul
        team.agg_foul_prob = sum(player.historical_stats.foul_prob for player in team.live_lineup)
        team.norm_foul_dist = [player.historical_stats.foul_prob / team.agg_foul_prob for player in team.live_lineup]

        # Probabilities for each active player of drawing a foul
        team.agg_FT_pg = sum(player.historical_stats['FT_pg'] for player in team.live_lineup)
        team.norm_freethrows_dist = [player['FT_pg'] / team.agg_FT_pg for player in team.live_lineup]

        # Probabilities of each active player of committing a turnover
        team.agg_turnover_prob= sum(player['TO_prob'] for player in team.live_lineup)
        team.norm_turnover_dist = [player['TO_prob']/team.agg_turnover_prob for player in team.live_lineup]

        # Probabilities for each active player to shoot a 2pt
        team.agg_2ptshots_per_minute = sum(player.historical_stats.two_pt_permin for player in team.live_lineup)
        team.norm_2ptshooting_dist = [player.historical_stats.two_pt_permin / team.agg_2ptshots_per_minute for player in team.live_lineup]
        
        # Probabilities for each active player to shoot a 3pt
        team.agg_3ptshots_per_minute = sum(player.historical_stats.three_pt_permin for player in team.live_lineup)
        team.norm_3ptshooting_dist = [player.historical_stats.three_pt_permin / team.agg_3ptshots_per_minute for player in team.live_lineup]

        # Probability of a 2pt shot
        team.shot_selection_2pt = team.agg_2ptshots_per_minute / (team.agg_2ptshots_per_minute + team.agg_3ptshots_per_minute)


    def update_game(self):
        '''
        Update: Possesssion, 
        '''
        pass

    def tipoff(self):
        ''' 50/50 tipoff for now'''
        return bool(random.binomial(1,0.5))

    def reset_game(self):
        pass

In [ ]:
class Simulation:
    def __init__(self, n_games:int, HomeTeam:str, AwayTeam:str):
        self.monte_iter = n_games
        self.home_team = HomeTeam
        self.away_team = AwayTeam
        self.game = Game(HomeTeam=self.home_team, AwayTeam=self.away_team)
        self.results = []
    
    def run_simulation(self):
        self.simulation_setup()
        for _ in range(self.monte_iter):
            self.game.simulate()
            self.results.append(self.game.get_results()) # Probably a better way to store the results of each monte carlo iteration to leverage matrix operations
            self.game.reset()
    def simulation_setup(self):
        ''' 
        When a simulatihon is started, all the player objects are initialized and the game object is initialized.
        1. Use the team names to query the database and get historical data on the players from both teams.
        
        '''
        
        

